This notebook shows how data from different Womens' Leagues are collected, using *Python* and *BeautyfulSoup*, and how this data will be loaded into a *Microsoft SQL Server* for later use in *Power BI*.

# Imports and global variables

In [1]:
import requests
import pandas as pd
import numpy as np
import re
import time
import pause, datetime

from bs4 import BeautifulSoup
from xPoints import *


leagues = [
    ['230', 'Liga-F'],
    ['189', 'Womens-Super-League'],
    ['193', 'Division-1-Feminine'],
    ['183', 'Frauen-Bundesliga'],
    ['208', 'Serie-A']
]

league_ids = pd.DataFrame(leagues, columns=['league_id', 'league_name'])
print(league_ids)

keys = [
    'stats',
    'shooting',
    'passing',
    #'passing_types', NOT INTERESTING FOR ME...
    'gca',
    'defense',
    'possession',
    'playingtime',
    'misc'
]

gk_keys = [
    'keepers',
    'keepersadv'
]

url_base = 'https://fbref.com/en/comps/%s/%s/%s'

# Create Requesting session
session = requests.Session()
print(datetime.datetime.now())

  league_id          league_name
0       230               Liga-F
1       189  Womens-Super-League
2       193  Division-1-Feminine
3       183    Frauen-Bundesliga
4       208              Serie-A
2022-12-11 10:23:33.628544


# Functions

In [2]:
def get_tables(url):

    res = session.get(url)
    # Updating cookies to avoid fbref blocking
    cookies = session.cookies.get_dict()
    cookies['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0'
    session.headers.update(cookies)
    
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    data = [pd.read_html(str(t), extract_links='body')[0] for t in soup.find_all('table')]
    return data


delete_colnames = [
    'Performance', 'Playing Time', 'Penalty Kicks', 'Expected', 'Standard', 'A', 
    'SCA', 'GCA', 'Starts', 'Subs' #, 'Team Success', 'Team Success (xG)'
]

def parse_multilevel_dataframe(data):
    colnames = []
    for (l0, l1) in list(data):
        if l0 in ['Per 90 Minutes']:
            colnames.append(l1 + '_90')
        else:
            if (l0 in delete_colnames) | (re.match(r'Unnamed.*', l0) != None):
                colnames.append(l1)
            else:
                colnames.append(l0 + '_' + l1)
    
    data = data.droplevel(0, axis=1)
    data.columns = colnames
    # Drop % columns and normalized by 90 minutes
    data = data[data.columns.drop(list(data.filter(regex='(%)|(90)')))]
    return data

def parse_link(id_url):
    link_regex = r'/en/([^/]+)/([^/]+)/.+'
    m = re.match(link_regex, str(id_url))
    if m:
        return m.groups()[1]
    else:
        return None

def parse_id_columns(data):
    for col in list(data):
        try:
            data[[col, col+'_id']] = pd.DataFrame(data[col].tolist(), index=data.index)
            data[col+'_id'] = data[col+'_id'].apply(lambda x: parse_link(x))
        except:
            data[col] = data[col].squeeze().copy()
            print('Error: ', col)
    data = data.drop_duplicates(keep=False).reset_index(drop=True)
    data.dropna(axis=1, inplace=True)
    return data

# Match Scores

In [3]:
matches = []
for idx, l in league_ids.iterrows(): 
    print('\nProcessing league ', l[1], ' matches...')

    url_scrape = url_base % (str(l[0]), 'schedule', l[1])
    tables = get_tables(url_scrape)
    m = tables[0]
    m = parse_id_columns(m)
    m['League_id'] = [l[0]] * m.shape[0]
    
    matches.append(m)
    req_time = datetime.datetime.now()
    # Timeout to avoid FBRef banning
    pause.until(req_time + datetime.timedelta(0,3.5))
    

matches = pd.concat(matches)
matches = matches[matches['Match Report'] == "Match Report"]
matches.rename(columns={'xG':'xGHome', 'xG.1':'xGAway', 'Match Report_id':'Match_id'}, inplace=True)

home_xp, away_xp = calculate_xpoints(matches=matches, num_simulations=50000, debug=False)
matches.loc[:, 'xPHome'] = home_xp
matches.loc[:, 'xPAway'] = away_xp

matches[['ScoreHome','ScoreAway']] = matches['Score'].str.split('–',expand=True)
matches.drop(columns=['Attendance', 'Venue', 'Referee', 'Match Report', 'Notes'], inplace=True)

matches.to_csv('datasets/matches.csv', index=False)
print('All matches proccessed')


Processing league  Liga-F  matches...

Processing league  Womens-Super-League  matches...

Processing league  Division-1-Feminine  matches...

Processing league  Frauen-Bundesliga  matches...

Processing league  Serie-A  matches...
All matches proccessed


# Stats by Teams and Players

In [4]:
teams_leagues = []
players_leagues = []

start_time = time.time()
for idx, l in league_ids.iterrows(): 
    print('\nProcessing league ', l[1], '\nScraping:', end='  ')
    teams = []
    players = []
    for k in keys:
        print(k, end='..  ')
        url_scrape = url_base % (str(l[0]), k, l[1])
        tables = get_tables(url_scrape)
        req_time = datetime.datetime.now()

        t, p = tables[0], tables[2]
        t, p = parse_multilevel_dataframe(t), parse_multilevel_dataframe(p)
        t, p = parse_id_columns(t), parse_id_columns(p)
        t['League_id'] = [l[0]] * t.shape[0]
        teams.append(t)
        players.append(p)
        # Timeout to avoid FBRef banning
        pause.until(req_time + datetime.timedelta(0,3.5))

    print('\n_______')
    # Drop NA and duplicated columns
    all_pl = pd.concat(players, axis=1).dropna()
    all_teams = pd.concat(teams, axis=1).dropna()
    teams_leagues.append(all_teams.loc[:,~all_teams.columns.duplicated()])
    players_leagues.append(all_pl.loc[:,~all_pl.columns.duplicated()])

print('Data scrapped in ', round(time.time() - start_time, 2), ' seconds')


Processing league  Liga-F 
Scraping:  stats..  shooting..  passing..  gca..  defense..  possession..  playingtime..  misc..  
_______

Processing league  Womens-Super-League 
Scraping:  stats..  shooting..  passing..  gca..  defense..  possession..  playingtime..  misc..  
_______

Processing league  Division-1-Feminine 
Scraping:  stats..  shooting..  passing..  gca..  defense..  possession..  playingtime..  misc..  
_______

Processing league  Frauen-Bundesliga 
Scraping:  stats..  shooting..  passing..  gca..  defense..  possession..  playingtime..  misc..  
_______

Processing league  Serie-A 
Scraping:  stats..  shooting..  passing..  gca..  defense..  possession..  playingtime..  misc..  
_______
Data scrapped in  189.63  seconds


## Data cleaning

In [5]:
# Merge different leagues
players_stats = pd.concat(players_leagues)

players_stats = players_stats[players_stats.columns.drop(list(players_stats.filter(regex=r'(Short*)|(Medium*)|(Long*)|(SCA_*)|(GCA_*)')))]
# Parse Minutes Played (1,000 -> 1000)
players_stats['Min'] = players_stats['Min'].apply(lambda x: int(x.replace(',', '')))

players_stats.rename(columns=lambda x: re.sub('\sTypes','',x), inplace=True)
players_stats.rename(columns=lambda x: re.sub('1/3','LastThird',x), inplace=True)
# 0,1,2
players_stats.rename(columns=lambda x: re.sub('\+/\-','Diff',x), inplace=True)
players_stats.rename(columns=lambda x: re.sub('\+','_plus_',x), inplace=True)
players_stats.rename(columns=lambda x: re.sub('\s+','',x), inplace=True)
# Playtime and misc
players_stats.rename(columns=lambda x: re.sub('/','_by_',x), inplace=True)
players_stats.rename(columns=lambda x: re.sub('Total_','Passes_',x), inplace=True)

players_stats.rename(columns=lambda x: re.sub('TeamSuccess(\s\(xG\))*','TS',x), inplace=True)

players_stats.drop(columns=['Rk', 'Matches', 'Matches_id', 'npxG_plus_xAG', 'G-PK', \
                            'Cmp', 'Mn_by_MP', 'Mn_by_Start', 'Compl', 'Mn_by_Sub', \
                            'G-PK', 'G_by_Sh', 'G_by_SoT', 'PKwon', 'xA', 'A-xAG', 'Prog', 'Tackles_TklW', \
                            'VsDribbles_Tkl', 'VsDribbles_Att', 'VsDribbles_Past', 'Tkl_plus_Int', \
                            'Dribbles_Mis', 'Dribbles_Dis',] ,\
                   inplace=True)
players_stats.rename(columns={'Tackles_Tkl':'Tkl', 'Blocks_Blocks':'Blocks', 'Touches_Touches':'Touches'}, inplace=True)

players_stats.iloc[:,5:] = players_stats.iloc[:,5:].fillna(0)
print(players_stats.shape)

players_stats.to_csv('datasets/all_players.csv', index=False)

(1332, 78)


In [6]:
# Merge different leagues
teams_stats = pd.concat(teams_leagues)
teams_stats.iloc[:,1:] = teams_stats.iloc[:,1:].fillna(0)

teams_stats = teams_stats[teams_stats.columns.drop(list(teams_stats.filter(regex=r'(Short*)|(Medium*)|(Long*)')))]
teams_stats.rename(columns=lambda x: re.sub('\sTypes','',x), inplace=True)
teams_stats.rename(columns=lambda x: re.sub('1/3','LastThird',x), inplace=True)
# 0,1,2
teams_stats.rename(columns=lambda x: re.sub('\+/\-','Diff',x), inplace=True)
teams_stats.rename(columns=lambda x: re.sub('\+','_plus_',x), inplace=True)
teams_stats.rename(columns=lambda x: re.sub('\s+','',x), inplace=True)
# Playtime and misc
teams_stats.rename(columns=lambda x: re.sub('/','_by_',x), inplace=True)
teams_stats.rename(columns=lambda x: re.sub('TeamSuccess(\s\(xG\))*','TS',x), inplace=True)
teams_stats.rename(columns=lambda x: re.sub('Total_','Passes_',x), inplace=True)

teams_stats.drop(columns=['npxG_plus_xAG', 'G-PK', \
                            'Cmp', 'Mn_by_MP', 'Mn_by_Start', 'Compl', 'Mn_by_Sub', \
                            'G-PK', 'G_by_Sh', 'G_by_SoT', 'PKwon', 'xA', 'A-xAG', 'Prog', 'Tackles_TklW', \
                            'VsDribbles_Tkl', 'VsDribbles_Att', 'VsDribbles_Past', 'Tkl_plus_Int', \
                            'Dribbles_Mis', 'Dribbles_Dis',] ,\
                   inplace=True)
teams_stats.rename(columns={'Tackles_Tkl':'Tkl', 'Blocks_Blocks':'Blocks', 'Touches_Touches':'Touches'}, inplace=True)

print(teams_stats.shape)
teams_stats.to_csv('datasets/all_teams.csv', index=False)

(62, 88)


# Goalkeeping

In [7]:
gk_teams_leagues = []
gk_players_leagues = []

start_time = time.time()
for idx, l in league_ids.iterrows(): 
    print('\nProcessing league ', l[1], '\nScraping:', end='  ')
    teams = []
    players = []
    for k in gk_keys:
        print(k, end=',    ')
        url_scrape = url_base % (str(l[0]), k, l[1])
        tables = get_tables(url_scrape)
        req_time = datetime.datetime.now()
        t, p = tables[0], tables[2]
        t, p = parse_multilevel_dataframe(t), parse_multilevel_dataframe(p)
        t, p = parse_id_columns(t), parse_id_columns(p)
        t['League_id'] = [l[0]] * t.shape[0]
        teams.append(t)
        p = p[p['Rk'] != 'Rk']
        players.append(p)
        # Timeout to avoid FBRef banning
        pause.until(req_time + datetime.timedelta(0,3.5))
    
    all_teams = pd.concat(teams, axis=1)
    all_pl = pd.concat(players, axis=1)
    gk_teams_leagues.append(all_teams.loc[:,~all_teams.columns.duplicated()])
    gk_players_leagues.append(all_pl.loc[:,~all_pl.columns.duplicated()])
    print('\n_______')

print('Data scrapped in ', round(time.time() - start_time, 2), ' seconds')


Processing league  Liga-F 
Scraping:  keepers,    keepersadv,    
_______

Processing league  Womens-Super-League 
Scraping:  keepers,    keepersadv,    
_______

Processing league  Division-1-Feminine 
Scraping:  keepers,    keepersadv,    
_______

Processing league  Frauen-Bundesliga 
Scraping:  keepers,    keepersadv,    
_______

Processing league  Serie-A 
Scraping:  keepers,    keepersadv,    
_______
Data scrapped in  41.47  seconds


In [8]:
gk_teams = pd.concat(gk_teams_leagues)
gk_players = pd.concat(gk_players_leagues) 

# Cleaning null rows, columns and colnames
gk_players.drop(columns=['Rk', 'Nation', 'Age', 'Born', 'Matches'], inplace=True)

# Parse Minutes Played (1,000 -> 1000)
gk_players['Min'] = gk_players['Min'].apply(lambda x: int(x.replace(',', '')))
gk_teams['Min'] = gk_teams['Min'].apply(lambda x: int(x.replace(',', '')))

gk_players.dropna(axis=1, inplace=True)
gk_players.dropna(axis=0, inplace=True)
gk_players.rename(columns={'CS':'Clean Sheets'}, inplace=True)
gk_players.rename(columns=lambda x: re.sub('\s+','',x), inplace=True)

gk_teams.rename(columns={'CS':'Clean Sheets'}, inplace=True)
gk_teams.rename(columns=lambda x: re.sub('\s+','',x), inplace=True)

gk_teams.to_csv('datasets/gk_teams.csv', index=False)
gk_players.to_csv('datasets/gk_players.csv', index=False)

In [9]:
print(datetime.datetime.now())

2022-12-11 10:28:32.515576
